# Prévision de la mortalité (France, 2000–2022)

## Ce projet étudie la mortalité de la population française sur la période 2000–2022.

## I - Construction des données:

 Les données exploitées dans cette étude proviennent de la Human Mortality Database
 (https://www.mortality.org), plateforme spécialisée dans la collecte et la diffusion de
 statistiques démographiques longitudinales. Après inscription, nous avons pu accéder à deux
 fichiers essentiels concernant la population française.

### 1 -  Prétraitement de la base des décès (Death counts):

La première base téléchargée concerne le **nombre annuel de décès** selon trois dimensions :

- **Year** : année civile *(t)*
- **Age** : âge exact *(x)*, variant de **0** à **110** ans
- **Sexe** : *(i = Female, Male)*, et **Total**

Après copie manuelle depuis la plateforme vers un **fichier texte** via *Notepad*, les données ont été importées dans **Excel**.  
Nous avons structuré la base selon les variables suivantes :

- **Year** : année civile *(t)*
- **Age** : âge exact *(x)*
- **D_F** : nombre de décès **féminins**
- **D_H** : nombre de décès **masculins**
- **D_Tot** : nombre de décès **totaux**

Cette base couvre initialement la période **1860–2022**, mais nous avons restreint l’échantillon à la période **2000–2022** pour des raisons de qualité, de modernité statistique et de pertinence actuarielle.


### 2 -  Prétraitement de la base des expositions (Exposure-to-risk)

La deuxième base contient les **expositions au risque de décès**, c’est-à-dire le nombre moyen d’individus vivants à chaque âge et pour chaque année.  
Les variables sont structurées de manière identique :

- **Year** : année civile *(t)*
- **Age** : âge exact *(x)*
- **E_F** : exposition **féminine**
- **E_H** : exposition **masculine**
- **E_Tot** : exposition **totale** (somme des deux sexes)

Les données ont été importées dans un second fichier Excel à partir d’un fichier texte brut, suivant le même protocole que pour les décès.  
La période analysée est également restreinte à **2000–2022**.


### 3 - Fusion des deux bases

Les deux bases ont été fusionnées par **jointure interne** sur les clés communes :

**Year = t**    et    **Age = x**

Le jeu de données final comporte donc les colonnes suivantes :

- **Year**, **Age**
- **E_F**, **E_H**, **E_Tot**
- **D_F**, **D_H**, **D_Tot**
